In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
import os

In [2]:
#set Java home
os.environ["JAVA_HOME"] = r"C:\Program Files\Java\jdk-18.0.2.1"

In [3]:
conf = SparkConf() \
    .setAppName("Example") \
    .setMaster("local") \
    .set("spark.driver.extraClassPath","C:/pyspark/*")

In [ ]:
sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)
spark

In [ ]:
df=spark.read.options(delimiter=",", header=True).csv(r"C:\Users\haq\OneDrive\Notebooks\data\AdvWorksData.csv")
df.show()

In [ ]:
df.printSchema()

In [ ]:
france = df.filter(df.saleterritory == "France").show(truncate=False)

In [ ]:
df1 = df[['productcategory','saleterritory','OrderDate','Sales']]
df1.show()

In [ ]:
saleterritory = df.groupBy('saleterritory').count()
print(saleterritory.show())

In [ ]:
df.createOrReplaceTempView("sales")
output =  spark.sql("SELECT * from sales where productsubcategory='Caps'")
output.show()

In [ ]:
output = spark.sql('SELECT * from sales WHERE `UnitPrice` > 20 AND OrderQuantity >= 10')
output.show()

In [ ]:
output = spark.sql('SELECT COUNT(*) as total, productcategory from sales GROUP BY productcategory')
output.show()

In [ ]:
dest_tbl = 'public."pyspark_sales_table"'
database = "AdventureWorks"
password = "demopass"#os.environ['PGPASS']
user = "etl"#os.environ['PGUID']

In [ ]:
df.write.mode("overwrite") \
    .format("jdbc") \
    .option("url", f"jdbc:postgresql://localhost:5432/{database}") \
    .option("dbtable", dest_tbl) \
    .option("user", user) \
    .option("password", password) \
    .option("driver",  "org.postgresql.Driver") \
    .save()